# Tests
Various tests performed on the files to explore the results coming from the model etc.

In [3]:
import pandas as pd
from src.utils import *
import IPython.display as ipd
from src.audio_utils import read_audio
import librosa

In [7]:
config_path = '../config/residualnet_train_config.yaml'
config = read_yaml(config_path)

In [10]:
df_eval = pd.read_csv(os.path.join('..', config['df_eval_path']))
file_eval = list(df_eval['path'])
label_eval = list(df_eval['label'])

# get the single file path
index = 2
file = file_eval[index]
label = label_eval[index]
print(f'Evaluating file {file} with label {label}')

Evaluating file /nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000024.flac with label 1


In [11]:
X, fs = read_audio(file)
print(f'File {0} is {X.size} samples long with fs={fs}')

File 0 is 29696 samples long with fs=16000


In [13]:
ipd.Audio(file)

In [14]:
s = librosa.core.stft(X, n_fft=2048, win_length=2048, hop_length=512, center=False)
s.shape

(1025, 55)

In [15]:
# checks on evaluation files list and scores obtained
print(f'Number of evaluation files is {len(file_eval)}')
print(f'Number of evaluation files labels is {len(label_eval)}')
print(f'The last 10 evaluated files are:\n {file_eval[len(file_eval)-10:len(file_eval)]}')

Number of evaluation files is 611829
Number of evaluation files labels is 611829
The last 10 evaluated files are:
 ['/nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_4999917.flac', '/nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_4999922.flac', '/nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_4999924.flac', '/nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_4999930.flac', '/nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_4999940.flac', '/nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_4999945.flac', '/nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_4999962.flac', '/nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_4999964.flac', '/nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_4999980.flac', '/nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_4999993.flac']


In [17]:
eval_scores_path = '../eval/scores_resnet_spec_eval.csv'
eval_scores_list = []

import csv
with open(eval_scores_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        if row:
            eval_scores_list.append(row[0])
            
print(f'The length of the score list is {len(eval_scores_list)}')

The length of the score list is 611829


In [18]:
eval_scores_list[0]

'/nas/public/dataset/asvspoof2021/ASVspoof2021_DF_eval/flac/DF_E_2000011.flac -9.027811050415039'

In [1]:
file_path = '../attacks/FGSM_data/p_audio/FGSM_DF_E_2000011_0.3.flac'

In [4]:
ipd.Audio(file_path)